In [1]:
"""
instantiate dataset
"""
from givernylocal.turbulence_dataset import *
from givernylocal.turbulence_toolkit import *
import numpy as np


auth_token = 'edu.gatech.rzeng42-bfe15f57'
dataset_title = 'isotropic1024coarse'
output_path = './giverny_output'

# instantiate the dataset.
dataset = turb_dataset(dataset_title = dataset_title, output_path = output_path, auth_token = auth_token)

In [2]:
"""
initialize getData parameters (except time and points)
"""
variable = 'velocity'
temporal_method = 'none'
spatial_method = 'lag8'
spatial_operator = 'field'

"""
example point distributions (2D plane, 3D box, random, time series) are provided below...
""";

In [3]:
def reverse_maximin(points):
    n = np.shape(points)[0]
    indices = np.zeros(n, dtype=int)
    lengths = np.zeros(n, dtype=float)
    dists = np.linalg.norm(points - points[0], axis=1)
    indices[-1] = 0
    lengths[0] = np.inf
    for i in range(n - 2, -1, -1):
        k = np.argmax(dists)
        indices[i] = k
        lengths[i] = dists[k]
        dists = np.minimum(dists, np.linalg.norm(points[k] - points, axis=1))
    return indices, lengths

def maximin(points):
    order = reverse_maximin(points)[0]
    return order[::-1]

In [ ]:
time = 0.0
time_end = 0.836
delta_t = 0.002
option = [time_end, delta_t]
nx = 10
ny = 10
nz = 10
n_points = nx * nz

x_points = np.linspace(0.0, 18 * np.pi / 512, nx, dtype = np.float64)
# y_points = np.pi / 2
y_points = np.linspace(0.0, 18 * np.pi / 512, ny, dtype = np.float64)
z_points = np.linspace(0.0, 18 * np.pi / 512, nz, dtype = np.float64)

points = np.array([axis.ravel() for axis in np.meshgrid(x_points, y_points, z_points, indexing = 'ij')], dtype = np.float64).T
order = maximin(points)
ordered_points = points[order]
result = []

while time_end < 10.056:
    print(time, time_end)
    cur = getData(dataset, variable, time, temporal_method, spatial_method, spatial_operator, ordered_points, option)
    result.extend(cur)
    time = round(time_end + delta_t, 3)
    time_end = round(time_end + 0.838, 3)
    option = [time_end, delta_t]

0.0 0.836

-----
getData is processing...



total time elapsed = 121.706 seconds (2.028 minutes)

query completed successfully.
-----
0.838 1.674

-----
getData is processing...

total time elapsed = 117.932 seconds (1.966 minutes)

query completed successfully.
-----
1.676 2.512

-----
getData is processing...

total time elapsed = 132.149 seconds (2.202 minutes)

query completed successfully.
-----
2.514 3.35

-----
getData is processing...

total time elapsed = 161.754 seconds (2.696 minutes)

query completed successfully.
-----
3.352 4.188

-----
getData is processing...

total time elapsed = 177.710 seconds (2.962 minutes)

query completed successfully.
-----
4.19 5.026

-----
getData is processing...

total time elapsed = 156.843 seconds (2.614 minutes)

query completed successfully.
-----
5.028 5.864

-----
getData is processing...

total time elapsed = 249.360 seconds (4.156 minutes)

query completed successfully.
-----
5.866 6.702

-----
getData is processing...

total time elapsed = 141.275 seconds (2.355 minutes)

qu

In [ ]:
# print(len(result[0]))
velocity = np.stack([df[['ux', 'uy', 'uz']].to_numpy() for df in result], axis=0)
# velocity = np.zeros((5028, 1000, 3), dtype = np.float64)
# for i in range(len(result)):
#     for j in range(len(result[i])):
#         velocity[i][j][0] = result[i].iloc[j]['ux']
#         velocity[i][j][1] = result[i].iloc[j]['uy']
#         velocity[i][j][2] = result[i].iloc[j]['uz']
# print(velocity)

In [9]:
print(velocity.shape)

(5028, 1000, 3)


In [10]:
import pickle
with open('10x10x10iso1024.pickle', 'wb') as f:
    pickle.dump(velocity, f)